# Differential Evolution Neural Network training

Using differential evolution to train neural network. The task is basically the problem of optimizing the function of 83 arguments (number depends on architecture), which are the weights and biases of neural network. 

In [33]:
from keras.utils import to_categorical
from scipy import optimize as op
import pandas as pd
import numpy as np

In [35]:
# Upload the Iris dataset from file. 
dataset = pd.read_csv('iris.data', header=0)

# Shuffling dataset to split it into train and test set, and avoid high variance.
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head(10)

,sepal length,sepal width,petal length,petal width,class
0,4.9,3.1,1.5,0.1,Iris-setosa
1,5.7,2.5,5.0,2.0,Iris-virginica
2,5.8,2.7,5.1,1.9,Iris-virginica
3,6.5,3.0,5.2,2.0,Iris-virginica
4,6.4,2.7,5.3,1.9,Iris-virginica
5,5.0,3.5,1.3,0.3,Iris-setosa
6,6.4,2.8,5.6,2.2,Iris-virginica
7,6.3,3.4,5.6,2.4,Iris-virginica
8,6.2,3.4,5.4,2.3,Iris-virginica
9,5.6,2.5,3.9,1.1,Iris-versicolor


In [36]:
# Adding bias unit.
dataset.insert(0, 'bias', 1)

# Mapping class names to numbers.
dataset['class'] = dataset['class'].map({'Iris-setosa':0, 
                                         'Iris-versicolor':1, 
                                         'Iris-virginica':2}).astype(int)
train = dataset[:100]
test = dataset[100:]

X = train.as_matrix(['bias',
                   'sepal length', 
                   'sepal width', 
                   'petal length', 
                   'petal width'])
y = train.as_matrix(['class'])
y = to_categorical(y)

X_test = test.as_matrix(['bias',
                   'sepal length', 
                   'sepal width', 
                   'petal length', 
                   'petal width'])
y_test = test.as_matrix(['class'])
y_test = to_categorical(y_test)

In [50]:
np.random.choice(X.shape[0], 3, replace=False)

array([72, 61, 31])

In [51]:
# Defining Rectified Linear Unit, which is used as actiation function here.
def relu(x):
    return x * (x > 0)

# Defining softmax, which is used in the output layer.
def softmax(a):
    s = np.exp(a - np.max(a))
    if a.ndim <3: return s/s.sum(0) 

# Defining Function to minimize.
def f(theta):
    # Reshaping 1D input weights to 2 2D matrix.    
    theta1 = theta[:40]
    bias1 = theta[40:50]
    theta2 = theta[50:80]
    bias2 = theta[80:]
    theta1 = theta1.reshape((10, 4))
    theta2 = theta2.reshape((3, 10))
    theta1 = np.column_stack((bias1, theta1))
    theta2 = np.column_stack((bias2, theta2))
    
    # Choosing a batch of 3 random samples.
    batch_idx = np.random.choice(X.shape[0], 10, replace=False)
    x_batch = X[batch_idx]
    y_batch = y[batch_idx]
    
    # Calculationg the loss via cross entropy.
    h = relu(theta1.dot(x_batch.T))
    h = np.row_stack((np.ones(h.shape[1]), h))
    out = softmax(theta2.dot(h))
    loss = -np.log((y_batch.T * out).sum(0))
    return loss.sum()

In [52]:
print type(f_test)
f(f_test)

<type 'numpy.ndarray'>


23.155324650957617

In [53]:
# bounds = [(-2., 2.)]*70
# op.differential_evolution(f, bounds, maxiter=50000)
bounds = [(-2., 2.)]*83
op.differential_evolution(f, bounds, maxiter=10000)

     fun: 0.0
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 12453009
     nit: 10000
 success: False
       x: array([ 1.30848932,  0.81764324,  1.80799548,  0.34838307, -0.07685203,
        1.67280683, -0.86301839, -1.0747434 ,  1.79727309, -0.71171146,
       -0.01800825, -1.16365216, -1.95096072,  0.62491266,  0.05381063,
        1.71313736, -0.39757288, -0.16410246,  1.90997168,  1.96311439,
       -1.3600024 , -1.30905888, -0.93432226,  1.01873819,  0.49787334,
       -0.97811568, -1.27805269,  1.76423449,  0.73181911,  0.0686799 ,
        1.25301359, -1.34329424,  0.38809433, -1.8002075 , -0.88237253,
        0.4462813 , -0.45414581,  1.50860079,  1.64726395,  1.59883219,
        1.33293314,  0.74163364,  0.59915971, -0.71005133, -0.1129822 ,
        1.35726647, -1.35842979,  0.40672924, -0.250284  ,  0.30929123,
       -1.46753851,  0.17188682,  0.81539141,  0.18308485, -1.29143111,
        1.35951747,  0.43201388, -0.9777995 , -1.04724225, -1.3436225 ,
 